In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, roc_auc_score,r2_score

In [ ]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [ ]:
PARAM = {
    'train_featus': Path("train_features.csv"),
    'train_labels': Path("train_labels.csv"),

    # For classification
    'target_label': "LABEL_Sepsis",
    'test_featus' : Path("test_features.csv"),
    
    'train_split': 0.8
}

df_meta = {
    'train': {
        'featus': pd.read_csv(PARAM['train_featus']),
        'labels': pd.read_csv(PARAM['train_labels']),
    },
    'test' : pd.read_csv(PARAM['test_featus'])
}

#print("Training features:", list(df_meta['train']['featus'].columns))
print("Training labels:", list(df_meta['train']['labels'].columns))

X = df_meta['train']['featus']

In [ ]:
X1 = X.groupby('pid').last()
X2 = X1.fillna(X1.median())
X = X2

y_all = df_meta['train']['labels'].set_index('pid')

print(F'num sample X: {len(X)}, y: {len(y)}')

X, y_all = X.align(y_all, axis=0, join='inner')
y_all  = y_all.astype(int)

In [ ]:
X_test = df_meta['test']
X_test1 = X_test.groupby('pid').last()
X_test2 = X_test1.fillna(X_test1.median())
X_test = X_test2

df_out = pd.DataFrame({'pid': X_test.index})

In [ ]:
# ls_clf = []
for i in np.arange(y_all.columns.get_loc(PARAM['target_label'])+1):
    y = y_all.iloc[:,i]
    #print("Classes:", dict(Counter(y)))

    #class_weight = dict(zip([0, 1], compute_class_weight('balanced', [0, 1], y)))
    print(list(y_all.columns)[i]) #,"Class weights:", class_weight)

    (X_train, X_valid, y_train, y_valid) = train_test_split(X, y, train_size=PARAM['train_split'], random_state=2)
    #print(F"Train/valid size: {len(X_train)}/{len(X_valid)}")

    clf = RandomForestClassifier(max_depth=12, random_state=2, n_estimators=100)#, class_weight=class_weight)
    clf.fit(X_train, y_train)
    
    #print("ROC~AUC score, train:", roc_auc_score(y_train, clf.predict_proba(X_train)[:, 1]))
    #print("ROC~AUC score, valid:", roc_auc_score(y_valid, clf.predict_proba(X_valid)[:, 1]))

    y_pred = clf.predict_proba(X_test)
    #print(y_pred[:, 1])
    
    df_out[list(y_all.columns)[i]] = y_pred[:,1]
#print(df_out.head())

In [ ]:
#ls_regr = []
for i in np.arange(y_all.columns.get_loc(PARAM['target_label'])+1,15):
    y = y_all.iloc[:,i]
    print(list(y_all.columns)[i])

    (X_train, X_valid, y_train, y_valid) = train_test_split(X, y, train_size=PARAM['train_split'], random_state=2)
    #print(F"Train/valid size: {len(X_train)}/{len(X_valid)}")

    regr = RandomForestRegressor(max_depth=12, random_state=0, n_estimators=100)
    regr.fit(X_train, y_train)
    
    y_pred = regr.predict(X_test)
    df_out[list(y_all.columns)[i]] = y_pred

print(df_out.head())

In [ ]:
df_out.to_csv('prediction.csv', index=False, float_format='%.3f', compression='zip')

irrelavant part

In [ ]:
    '''print('')
    ls_clf.append(roc_auc_score(y_valid, clf.predict_proba(X_valid)[:, 1]))
print(ls_clf)

s1 = np.mean(ls_clf)
print(s1)'''

    '''print("R2 score, train:", 0.5 + 0.5 * r2_score(y_train, regr.predict(X_train)))
    print("R2 score, valid:", 0.5 + 0.5 * r2_score(y_valid, regr.predict(X_valid)))
    print('')
    ls_regr.append(0.5 + 0.5 * r2_score(y_valid, regr.predict(X_valid)))
print(ls_regr)
s2 = np.mean(ls_regr)
print(s2)'''

In [ ]:
print('avg:',(s1+s2)/2)

In [ ]:
from score_submission import get_score

filename = 'sample.zip'
filename_sub = 'prediction.zip'

df_true = pd.read_csv(filename)
df_submission = pd.read_csv(filename_sub)

print('My score with sample.zip as groundtruth', get_score(df_true, df_submission))